In [5]:
import allel
import loompy
import numpy as np
import pandas as pd
import pysam
import seaborn as sns
import matplotlib.pyplot as plt
import math
import itertools
from collections import Counter
from collections import defaultdict

import mb_funcs as mb
%load_ext autoreload
%autoreload 2

plt.rcParams['xtick.labelsize'] = 20
plt.rcParams['ytick.labelsize'] = 20
plt.rcParams['axes.labelsize'] = 20

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
cnp_file = '/n/fs/ragr-data/datasets/mskcc_iacobuzio/organoid/bams/hatchet/cn_profiles.tsv'
mskcc_loomfile = 'IR125_P8_IGO_11988_C_4.cells.loom'
# mskcc_loomfile = '/n/fs/ragr-data/datasets/mskcc_iacobuzio/RA17_22/tapestri/RA17_22_P1_IGO_11988_B_2/11988_B_2.RA17_22_P1.cells.loom'

df_cnp = pd.read_csv(cnp_file, sep='\t')
ds = loompy.connect(mskcc_loomfile)
df_unique_pos = mb.get_position_dataframe_from_loom(ds)

In [86]:
ds.close()

In [16]:
df_unique_pos

,pos,amplicon,chrom,ref,alt,ref_len,alt_len,normal,index
7884,27057731,AMPL50980,1,A,G,1,1,True,0
7889,27057736,AMPL50980,1,C,A,1,1,True,2
7890,27057737,AMPL50980,1,A,ATCACCTCCATCT,1,13,False,4
7895,27057742,AMPL50980,1,C,A,1,1,True,6
7898,27057745,AMPL50980,1,T,TAGGAGATAACAC,1,13,False,7
...,...,...,...,...,...,...,...,...,...
27589,76777891,AMPL70504,X,T,C,1,1,True,42400
27590,76777892,AMPL70504,X,T,,1,0,False,42401
27591,76777893,AMPL70504,X,T,C,1,1,True,42402
27592,76777894,AMPL70504,X,C,T,1,1,True,42403


In [73]:
cnp1_list = []
cnp2_list = []
for idx, row in df_unique_pos.iterrows():
    curr_pos = row['pos']
    curr_chrm = row['chrom']
    
    if curr_chrm != 'X':
        curr_chrm = int(curr_chrm)
        df_select = df_cnp[(df_cnp['#CHR'] == curr_chrm) & (df_cnp['START'] <= curr_pos) & (df_cnp['END'] >= curr_pos)][['cn_clone1', 'cn_clone2']]
#         df_select = df_cnp[(df_cnp['START'] <= curr_pos) & (df_cnp['END'] >= curr_pos)]
        if len(df_select) == 0:
            cnp1_list.append(np.nan)
            cnp2_list.append(np.nan)
        elif len(df_select) == 1:
            cnp1_list.append(df_select['cn_clone1'].values[0])
            cnp2_list.append(df_select['cn_clone2'].values[0])            
        else:
            print(row, 'more than one matches', sep='\n')
            break
    else:
        cnp1_list.append(np.nan)
        cnp2_list.append(np.nan)    

In [74]:
df_unique_pos['cn_clone1'] = cnp1_list
df_unique_pos['cn_clone2'] = cnp2_list

In [78]:
df_unique_pos['cn_clone1'].isna().sum()

31723

In [79]:
df_unique_pos['cn_clone1'].count()

4393

In [80]:
df_unique_pos['cn_clone2'].count()

4393

In [87]:
df_cnp

,#CHR,START,END,SAMPLE,cn_normal,u_normal,cn_clone1,u_clone1,cn_clone2,u_clone2
0,1,1,1900210,Tumor1,1|1,0,2|0,0.75,2|2,0.25
1,1,12781309,15909873,Tumor1,1|1,0,2|0,0.75,2|2,0.25
2,1,16631374,17658040,Tumor1,1|1,0,2|0,0.75,2|2,0.25
3,1,132010540,144854023,Tumor1,1|1,0,2|0,0.75,2|2,0.25
4,1,144946880,149973911,Tumor1,1|1,0,2|0,0.75,2|2,0.25
5,1,167215303,168908003,Tumor1,1|1,0,2|0,0.75,2|2,0.25
6,1,214656371,218657421,Tumor1,1|1,0,2|0,0.75,2|2,0.25
7,1,227172920,227695917,Tumor1,1|1,0,2|0,0.75,2|2,0.25
8,10,42300000,51770618,Tumor1,1|1,0,2|0,0.75,2|2,0.25
9,11,970390,3381607,Tumor1,1|1,0,2|0,0.75,2|2,0.25


In [81]:
df_unique_pos_cn = df_unique_pos[df_unique_pos['cn_clone1'] == df_unique_pos['cn_clone1']]

In [82]:
df_unique_pos_cn

,pos,amplicon,chrom,ref,alt,ref_len,alt_len,normal,index,cn_clone1,cn_clone2
27595,77645600,AMPL87274,3,A,G,1,1,True,772,2|0,2|2
27618,77645623,AMPL87274,3,T,C,1,1,True,773,2|0,2|2
27619,77645624,AMPL87274,3,G,A,1,1,True,774,2|0,2|2
27673,77645678,AMPL87274,3,A,G,1,1,True,775,2|0,2|2
27699,77645704,AMPL87274,3,T,C,1,1,True,777,2|0,2|2
...,...,...,...,...,...,...,...,...,...,...,...
4582,12032538,AMPL87379,17,C,,1,0,False,33272,2|0,2|2
4584,12032540,AMPL87379,17,C,,1,0,False,33273,2|0,2|2
4585,12032541,AMPL87379,17,C,T,1,1,True,33274,2|0,2|2
4586,12032542,AMPL87379,17,GC,G,2,1,False,33275,2|0,2|2


In [83]:
df_unique_pos_cn['cn_clone1'].value_counts()

2|1    2629
2|0    1764
Name: cn_clone1, dtype: int64

In [85]:
df_unique_pos_cn['cn_clone2'].value_counts()

2|1    2629
2|2    1764
Name: cn_clone2, dtype: int64